In [1]:
%matplotlib inline

#- import required python libraries
import netCDF4
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import glofrim.glofrim_lib as glib
from os.path import join, isfile, abspath, dirname, basename, normpath
from configparser import ConfigParser
from glofrim.glofrim_lib import configread

In [2]:
from glofrim import Glofrim, PCR, LFP
cbmi = Glofrim()

In [3]:
#- provide file with local paths to model objects
env_fn = r'../../environment.env'
config = configread(env_fn)
root_dir = config.get('models', 'root_dir')

In [4]:
#- convert model configs to dictionary
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)

OrderedDict([(u'engines',
              OrderedDict([('CMF',
                            u'/home/jannis/Programmes/cama-flood_bmi_v3.6.2/src/libcama.so'),
                           ('DFM',
                            u'/home/jannis/Programmes/DFLOWFM/lib/libdflowfm.so'),
                           ('LFP',
                            u'/home/jannis/Programmes/lisflood-bmi-v5.9/liblisflood.so')])),
             (u'models',
              OrderedDict([('root_dir',
                            u'/home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe')]))])

In [5]:
#- provide file with coupling settings
# config_fn = join(root_dir, 'glofrim_PCR2CMF.ini')
# config_fn = join(root_dir, 'glofrim_PCR2DFM.ini')
# config_fn = join(root_dir, 'glofrim_WFL2DFM.ini')
config_fn = join(root_dir, 'glofrim_PCR2LFP.ini')
config_fn

u'/home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/glofrim_PCR2LFP.ini'

In [6]:
#- provide locations where to extract simulated discharge
x_obs_coord = 10.89 # set to coords of Neu-Darchau
y_obs_coord = 53.23

In [7]:
#- provide start and end date
t_start = datetime(2000,1,1)
t_end = datetime(2000,1,3)
t_start_test = cbmi.set_start_time(t_start)
try:
    t_start == t_start_test
except:
    sys.exit('start time differ with set_var and get_var')

In [8]:
#- create coupled GLOFRIM model and initialize configs
try:
    cbmi.initialize_config(config_fn, env_fn=env_fn)
except Exception as e:
    print e
    sys.exit('configs could not be initialized')

2018-07-24 11:14:05,224 - PCR - INFO - Config initialized
2018-07-24 11:14:05,232 - LFP - INFO - Config initialized


In [9]:
#- create coupled GLOFRIM model and initialize models
try:
    cbmi.initialize_model()
except Exception as e:
    print e
    sys.exit('models could not be initialized')

2018-07-24 11:14:05,243 - PCR - INFO - Getting rgrid info based on landmask_elbe_30min.map
2018-07-24 11:14:05,246 - PCR - INFO - Getting drainage direction from ldd_elbe_30min.map
2018-07-24 11:14:05,248 - LFP - INFO - Getting rgrid info based on LFP_Elbe_DEM_2km.asc
2018-07-24 11:14:05,309 - PCR - WARNING - /home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/PCR_Elbe/setup_PCR_30min_Elbe_glofrim.ini file overwritten
2018-07-24 11:14:05,311 - PCR - INFO - Ini file written to /home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/PCR_Elbe/setup_PCR_30min_Elbe_glofrim.ini
2018-07-24 11:14:05,315 pcrglobwb_bmi_v203.configuration INFO Model run started at 2018-07-24 11:14:05.312237
2018-07-24 11:14:05,317 pcrglobwb_bmi_v203.configuration INFO Logging output to /home/jannis/PhD/code/GLOFRIMplusCAMA/experiment_GLOFRIM2/PCR/log/setup_PCR_30min_Elbe.ini_2018-07-24T11:14:05.312237.log
2018-07-24 11:14:05,318 pcrglobwb_bmi_v203.configuration INFO Note that avgSurfaceWaterIn

1D nodes found outside of valid 2D domain


2018-07-24 11:14:05,611 pcrglobwb_bmi_v203.meteo INFO No forcing downscaling is implemented.
2018-07-24 11:14:13,553 pcrglobwb_bmi_v203.deterministic_runner INFO Adjusting some model parameters based on given values in the ini/configuration file.
2018-07-24 11:14:13,554 pcrglobwb_bmi_v203.deterministic_runner INFO 

Multiplier values used: 
For minSoilDepthFrac           : 1.0
For kSat (log-scale)           : 0.0
For recessionCoeff (log-scale) : 1.0
For storCap                    : 1.25
For degreeDayFactor            : 1.0
For refPotET                   : 1.0

2018-07-24 11:14:13,556 pcrglobwb_bmi_v203.pcrglobwb_bmi INFO Model initialized. Spin-up might be required.
2018-07-24 11:14:13,558 - PCR - INFO - Model initialized
2018-07-24 11:14:13,558 PCR INFO Model initialized
2018-07-24 11:14:13,560 - LFP - WARNING - /home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/LFP_data/LFPmodel_Elbe_glofrim.par file overwritten
2018-07-24 11:14:13,560 LFP WARNING /home/jannis/PhD/code/G

In [10]:
#- get corresponding indices for models
obs_ind_PCR = cbmi.index(x_obs_coord, y_obs_coord, 'PCR', in1d=False)
print obs_ind_PCR
obs_ind_LFP = cbmi.index(x_obs_coord, y_obs_coord, 'LFP', in1d=True)
print obs_ind_LFP

[67]
[405]


In [11]:
#- update models until end time is reached
#- also, read out discharge data at specified observation point
try:
    while t_start < t_end:
        cbmi.update()
        q_sim_PCR = cbmi.get_value_at_indices('PCR.discharge', obs_ind_PCR)
        q_sim_LFP = cbmi.get_value_at_indices('LFP.Qx', obs_ind_DFM) 
        print q_sim_PCR, q_sim_LFP
except Exception as e:
    print e
    sys.exit('something is going wrong in updating - please check!')

2018-07-24 11:14:13,791 pcrglobwb_bmi_v203.pcrglobwb INFO reading forcings for time 2000-01-01
2018-07-24 11:14:13,959 pcrglobwb_bmi_v203.pcrglobwb INFO updating model to time 2000-01-01


/home/jannis/Programmes/lisflood-bmi-v5.9/liblisflood.so: undefined symbol: set_var


SystemExit: something is going wrong in updating - please check!

/home/jannis/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#- finalize all models
try:
    cbmi.finalize()
except Exception as e:
    print e
    sys.exit('models could not be finalized')